# This notebooks open TPXO product, and interpolate it from regular latlon1/6° to llc1080 grid.

In [1]:
import xarray as xr
import numpy as np
import pyTMD.io
from xmitgcm import open_mdsdataset
import matplotlib.pyplot as plt

In [10]:
grid_file = '../TMD/TPXO9.1/DATA/grid_tpxo9'
ocean_file = '../TMD/TPXO9.1/DATA/h_tpxo9.v1'
#load_file = '../TMD/TPXO9.1/DATA/h_load_file' # not used here, this file contains the SAL extracted from the TPXO product

ds_tpxo= pyTMD.io.OTIS.open_dataset( #Open TPXO using pyTMD
        grid_file=grid_file, 
        model_file=ocean_file, 
        projection=4326,
        type='z',
        grid='OTIS'
    )

In [11]:
print(ds_tpxo)

<xarray.Dataset> Size: 299MB
Dimensions:     (y: 1081, x: 2160)
Coordinates:
  * y           (y) float64 9kB -90.0 -89.83 -89.67 -89.5 ... 89.67 89.83 90.0
  * x           (x) float64 17kB 0.1667 0.3333 0.5 0.6667 ... 359.7 359.8 360.0
Data variables: (12/17)
    bathymetry  (y, x) >f4 9MB 0.0 0.0 0.0 0.0 ... 4.177e+03 4.177e+03 4.177e+03
    mask        (y, x) int32 9MB 0 0 0 0 0 0 0 0 0 0 0 ... 1 1 1 1 1 1 1 1 1 1 1
    m2          (y, x) complex64 19MB (nan+nanj) ... (0.01560137-0.049397722j)
    s2          (y, x) complex64 19MB (nan+nanj) ... (-0.013529291-0.018836113j)
    n2          (y, x) complex64 19MB (nan+nanj) ... (0.015766058+0.0053001363j)
    k2          (y, x) complex64 19MB (nan+nanj) ... (0.00013639421-0.0067234...
    ...          ...
    mf          (y, x) complex64 19MB (nan+nanj) ... (-0.020163985+0.016151741j)
    m4          (y, x) complex64 19MB (nan+nanj) ... (8.278735e-08-2.6768595e...
    mn4         (y, x) complex64 19MB (nan+nanj) ... (-2.4361745e-07-2.93

# Interpolation

### Import grid files

In [12]:
"""
I need XC and YC. The simplest way I found was to import 1point of time series, and extract the grid from there. 
The Eta point is from the first run of Dan with hourly output, but it doesn't matter anyway, since I only need the grid.
It's very quick, but the inconvenient is that I need to store the grid folder 'run_dan' locally.
There is surely a smarter way.
"""
ds_eta1 = open_mdsdataset('../Eta_1point', 
                        grid_dir='../run_dan', 
                         prefix={'Eta'},
                         read_grid=True,
                         geometry="llc")
XC = ds_eta1['XC'].data
YC = ds_eta1['YC'].data
ds_eta1 = ds_eta1.rename({'face':'tile'})

/Users/hugopp/miniconda3/envs/mitgcmenv_clean/lib/python3.11/site-packages/xmitgcm/mds_store.py:289: FutureWarning: In a future version, xarray will not decode the variable 'time' into a timedelta64 dtype based on the presence of a timedelta-like 'units' attribute by default. Instead it will rely on the presence of a timedelta64 'dtype' attribute, which is now xarray's default way of encoding timedelta64 values.
To continue decoding into a timedelta64 dtype, either set `decode_timedelta=True` when opening this dataset, or add the attribute `dtype='timedelta64[ns]'` to this variable on disk.
To opt-in to future behavior, set `decode_timedelta=False`.
  ds['time'] = xr.decode_cf(ds[['time']])['time']


In [13]:
import numpy as np
import xarray as xr
from scipy.interpolate import RegularGridInterpolator

def interpolate_tpxo_to_llc(ds_tpxo, XC, YC):
    """
    Interpolates pyTMD/OTIS TPXO dataset to LLC tiles.
    """
    constituents = [v for v in ds_tpxo.data_vars if np.iscomplexobj(ds_tpxo[v])] # Identify tidal constituents (exclude bathy/mask/etc)

    ds_tpxo = ds_tpxo.sortby(['y', 'x']) #Extract and sort coordinates
    tpxo_lon = ds_tpxo.x.values
    tpxo_lat = ds_tpxo.y.values
    
    XC_norm = XC % 360 #Converts LLC lon convention (-180,180) to TPXO format (0,360)
    
    n_tiles, ny, nx = XC.shape #(13,1080,1080)
    ds_vars = {}

    for const in constituents: #Sum over the constituents in TPXO
        print("Interpolating "+const+' constituent')
        data = ds_tpxo[const].values
        data = np.nan_to_num(data) #Replace nans with 0 for interpolation
        #These functions interpolate TPXO values on each longitude and latitude 
        interp_r = RegularGridInterpolator((tpxo_lat, tpxo_lon), data.real, 
                                          method='linear', bounds_error=False, fill_value=0)
        interp_i = RegularGridInterpolator((tpxo_lat, tpxo_lon), data.imag, 
                                          method='linear', bounds_error=False, fill_value=0)
        # Tile arrays
        tile_real = np.zeros((n_tiles, ny, nx), dtype='float32')
        tile_imag = np.zeros((n_tiles, ny, nx), dtype='float32')
        
        for tile in range(n_tiles):
            pts = np.column_stack((YC[tile].ravel(), XC_norm[tile].ravel())) # Returns a list of coordinates (lat,lon)
            tile_real[tile] = interp_r(pts).reshape(ny, nx) #Return interpolated values for each cell of the tile 
            tile_imag[tile] = interp_i(pts).reshape(ny, nx)
            ds_vars[f"{const}_real"] = (("tile", "j", "i"), tile_real) #stores in the tile arrays.
            ds_vars[f"{const}_imag"] = (("tile", "j", "i"), tile_imag)
            
    ds_out = xr.Dataset( #final dataset
        data_vars=ds_vars,
        coords={
            "tile": np.arange(n_tiles),
            "j": np.arange(ny),
            "i": np.arange(nx),
            "XC": (("tile", "j", "i"), XC),
            "YC": (("tile", "j", "i"), YC),
        }
    )
    return ds_out

In [14]:
XC_wrapped = np.where(XC < 0, XC + 360, XC) #Ensure XC is in the 0-360 range if TPXO is 0-360
ds_tpxo_llc= interpolate_tpxo_to_llc(ds_tpxo, XC_wrapped, YC)

Interpolating m2 constituent


/Users/hugopp/miniconda3/envs/mitgcmenv_clean/lib/python3.11/site-packages/dask/array/core.py:1733: FutureWarning: The `numpy.column_stack` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  warnings.warn(


Interpolating s2 constituent
Interpolating n2 constituent
Interpolating k2 constituent
Interpolating k1 constituent
Interpolating o1 constituent
Interpolating p1 constituent
Interpolating q1 constituent
Interpolating mm constituent
Interpolating mf constituent
Interpolating m4 constituent
Interpolating mn4 constituent
Interpolating ms4 constituent
Interpolating 2n2 constituent
Interpolating s1 constituent


In [15]:
ds_tpxo_llc = ds_tpxo_llc.astype(np.float32)

# 2. Define compression encoding for each data variable
encoding = {var: {'zlib': True, 'complevel': 5} for var in ds_tpxo_llc.data_vars}

# 3. Save to NetCDF
ds_tpxo_llc.to_netcdf('../tpxo_interpolated/tpxo_llc1080_interpolated_v2.nc', encoding=encoding)


In [16]:
print(ds_tpxo_llc)

<xarray.Dataset> Size: 2GB
Dimensions:   (tile: 13, j: 1080, i: 1080)
Coordinates:
  * tile      (tile) int64 104B 0 1 2 3 4 5 6 7 8 9 10 11 12
  * j         (j) int64 9kB 0 1 2 3 4 5 6 ... 1073 1074 1075 1076 1077 1078 1079
  * i         (i) int64 9kB 0 1 2 3 4 5 6 ... 1073 1074 1075 1076 1077 1078 1079
    XC        (tile, j, i) float32 61MB dask.array<chunksize=(1, 1080, 1080), meta=np.ndarray>
    YC        (tile, j, i) >f4 61MB dask.array<chunksize=(1, 1080, 1080), meta=np.ndarray>
Data variables: (12/30)
    m2_real   (tile, j, i) float32 61MB 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    m2_imag   (tile, j, i) float32 61MB 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    s2_real   (tile, j, i) float32 61MB 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    s2_imag   (tile, j, i) float32 61MB 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    n2_real   (tile, j, i) float32 61MB 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    n2_imag   (tile, j, i) float32 61MB 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    